In [1]:
import json
import numpy as np
from numpy import log,sqrt,pi
import itertools

## Define auxiliary functions

In [ ]:
vev = 246.
yt = 172./(vev/np.sqrt(2))
yb = 4.7/(vev/np.sqrt(2))
yta = 1.77/(vev/np.sqrt(2))
d = 'dummyIndex' # define dummy index

def Delta(a,b):

    # Dummy indices are treated as strings,
    # so we need to deal with it separately.
    # Assume it always return 1
    
    if isinstance(a,int) and isinstance(b,int):
        if a == b:
            return 1
        else:
            return 0
    else:
        return 1


# def yMatrix(a,b,val):

#     if isinstance(a,int):
#         x = a
#         y = b
#     else:
#         x = b
#         y = a
#     if x == 3:
#         if isinstance(y,int) and y == 3:
#             return val
#         elif isinstance(y,str): # If string, it is a dummy index
#             return 1
    
#     return 0

# def yu(a,b):
#     return  yMatrix(a,b,yt)

# def yd(a,b):
#     return  yMatrix(a,b,yb)

# def ye(a,b):
#     return  yMatrix(a,b,yta)

# def LF(masses,indices):

#     return 1.0


## Load Coefficient Expressions

In [3]:
with open('coeffs_python.json', 'r') as f:
    exprDict = json .load(f)

In [4]:
funcDict = {}
for coeff in exprDict:
    funcDict[coeff] = {'indices' : exprDict[coeff]['indices']}
    indices = ','.join(exprDict[coeff]['indices'])
    expr = exprDict[coeff]['expr']
    if indices.strip():
        exec(f"def {coeff}({indices},mST,mChi,yDM2,yDM4,g1,g2,g3): return {expr}")
    else:
        exec(f"def {coeff}(mST,mChi,yDM2,yDM4,g1,g2,g3): return {expr}")
    funcDict[coeff]['func'] = locals()[coeff]

### SM parameters

In [18]:
aEW = 1.0/127.9
MZ = 91.18
Gf = 1.166370e-05
aS = 0.13
MW = sqrt(MZ**2/2+sqrt(MZ**4/4-pi/sqrt(2)*aEW/Gf*MZ**2))
ee = np.sqrt(4*pi*aEW)
sw2 = 1.0 - (MW/MZ)**2 
cw = np.sqrt(1-sw2)
sw = np.sqrt(sw2)
g1 = ee/cw
g2 = ee/sw
g3 = np.sqrt(4*pi*aS)

### BSM Parameters

In [ ]:
mST = 500.0
mChi = 400.0

### Compute all coefficients

In [62]:
all_indices = [1,2,3]
results = []
for coeff in funcDict:
    indices = funcDict[coeff]['indices']
    for index_values in itertools.product(*((all_indices for i in range(len(indices))))):
        coeff_0 = float(funcDict[coeff]['func'](*index_values,mST,mChi,0,0,g1,g2,g3)) # term independent of yDM
        coeff_2 = float(funcDict[coeff]['func'](*index_values,mST,mChi,1,0,g1,g2,g3) )- coeff_0 # term scaling as  yDM^2
        coeff_4 = float(funcDict[coeff]['func'](*index_values,mST,mChi,0,1,g1,g2,g3)) - coeff_0 # term scaling as  yDM^4
        index_str = str(index_values).replace(',','').strip()
        results.append([coeff,index_str,coeff_0,coeff_2,coeff_4])
        # print(indices,index_values)
        # print(f'{coeff} = {funcDict[coeff]['func'](*index_values,mST,mChi,yDM,g1,g2,g3):1.3e}')

#### Remove zero entries

In [63]:
resultsNonZero = [tuple(r) for r in results if sum(r[-3:-1]) != 0.0]
print(len(resultsNonZero))
resultsNonZero = np.array(resultsNonZero,dtype=[('coeff','<U10'),('indices','<U15'),('C0','<f8'),('C2','<f8'),('C4','<f8')])

211


### Save results

In [66]:
header = f'# SM parameters:\n#  g1 = {g1:1.2e}, g2 = {g2:1.2e}, g3 = {g3:1.2e}, yt = {yt:1.2e} , yb = {yb:1.2e}, yta = {yta:1.2e}'
header += f'\n# UV parameters:\n#  mST = {mST:1.2f}, mChi = {mChi:1.2f}'
header += '\n# The total value for the coefficient is given by C0 + yDM^2*C2 + yDM^4*C4, where yDM is the BSM coupling'
header += '\nCoefficient,Indices,C0 (1/GeV^2),C2 (1/GeV^2),C4 (1/GeV^2)'
np.savetxt(f'coeff_values_mST_{mST:1.0f}_mChi_{mChi:1.0f}.csv',resultsNonZero,
           comments='',
           header=header,fmt='%s,%s,%1.3e,%1.3e,%1.3e')

In [14]:
coeff = None
for r in resultsNonZero:
    if r[0] != coeff:
        coeff = r[0]
        print(coeff)
    print('   ',r[1],f'{r[2]:1.3e}',f'{r[3]:1.3e}',f'{r[4]:1.3e}',f'{sum(r[2:5]):1.3e}')

cdd
    (1, 1, 1, 1) -1.030e-10 0.000e+00 0.000e+00 -1.030e-10
    (1, 1, 2, 2) 5.150e-11 0.000e+00 0.000e+00 5.150e-11
    (1, 1, 3, 3) 5.150e-11 0.000e+00 0.000e+00 5.150e-11
    (1, 2, 2, 1) -1.545e-10 0.000e+00 0.000e+00 -1.545e-10
    (1, 3, 3, 1) -1.545e-10 0.000e+00 0.000e+00 -1.545e-10
    (2, 1, 1, 2) -1.545e-10 0.000e+00 0.000e+00 -1.545e-10
    (2, 2, 1, 1) 5.150e-11 0.000e+00 0.000e+00 5.150e-11
    (2, 2, 2, 2) -1.030e-10 0.000e+00 0.000e+00 -1.030e-10
    (2, 2, 3, 3) 5.150e-11 0.000e+00 0.000e+00 5.150e-11
    (2, 3, 3, 2) -1.545e-10 0.000e+00 0.000e+00 -1.545e-10
    (3, 1, 1, 3) -1.545e-10 0.000e+00 0.000e+00 -1.545e-10
    (3, 2, 2, 3) -1.545e-10 0.000e+00 0.000e+00 -1.545e-10
    (3, 3, 1, 1) 5.150e-11 0.000e+00 0.000e+00 5.150e-11
    (3, 3, 2, 2) 5.150e-11 0.000e+00 0.000e+00 5.150e-11
    (3, 3, 3, 3) -1.030e-10 0.000e+00 0.000e+00 -1.030e-10
ced
    (1, 1, 1, 1) -3.753e-14 0.000e+00 0.000e+00 -3.753e-14
    (1, 1, 2, 2) -3.753e-14 0.000e+00 0.000e+00 -3.753e-14
 

In [8]:
exprDict['cuH']

{'indices': ['i1', 'i2'],
 'expr': 'yDM2*yt**3*(2*mChi**6 - 6*mChi**4*mST**2*log(mChi**2/mST**2) + 3*mChi**4*mST**2 - 6*mChi**2*mST**4 + mST**6)*Delta(3, d)**2*Delta(3, i1)*Delta(3, i2)/(192*pi**2*(mChi**2 - mST**2)**4)'}

In [9]:
Delta(3,d)

1